# Exercise 10: PCA - Solutions
In this exercise, we will implement and see the workings of a dimensionality reduction technique: Prinical Component Analysis (PCA).

We will also compare it to Fisher's Linear Discriminant Analysis (LDA) on MNIST.

Note: these solutions are partial as you will have to implement PCA for your project.

In [ ]:
# good to import few packages
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colormaps
from sklearn import datasets
from sklearn.datasets import fetch_olivetti_faces

# 1 Toy Dataset
Let see the PCA results on a toy dataset: `iris`.

In [ ]:
# load iris dataset
iris = datasets.load_iris()
data = iris['data'].astype(np.float32)
labels = iris['target'] 
cls_names = iris['target_names']

We will first visualize the trends of different features together. One can see that one class is well separated from the others.

In [ ]:
plt.figure(figsize=(12,12))
count = 1
colors = np.array([[0.85, 0.85, 0], [0, 0.5, 0], [0.25, 0.25, 1]])
for i in range(4):
    for j in range(4):
        plt.subplot(4, 4, count)
        for ind, name in enumerate(cls_names):
            filtered_class = labels == ind
            plt.scatter(data[filtered_class,i], data[filtered_class,j], c=colors[ind,None], label=name)
        plt.xlabel(f'feature_{i}')
        plt.ylabel(f'feature_{j}')
        plt.legend()
        count +=1

# 2 PCA
In the Iris dataset, we have 4 features per data point. Let's now try to reduce the dimensionality from $D=4$ to $d=2$ using PCA. 
As seen in class, for a dataset $\mathbf{X}\in \mathbb{R}^{N\times D}$ and for a 1D projection ${\bf w}_{1}$, PCA solves the following optimization problem
$$\begin{align}
    \max_{\mathbf{w}_{1}} \mathbf{w}_{1}^T\mathbf{C}\mathbf{w}_{1}\\
    s.t. ~~~~~ \mathbf{w}_{1}^T\mathbf{w}_{1} = 1
\end{align}$$
   
 where $\mathbf{C}$ is the data covariance matrix
$$\begin{align}
         \mathbf{C} &= \frac{1}{N}\sum_{i=0}^{N-1}(\mathbf{x}_i-\mathbf{\bar{x}})(\mathbf{x}_i-\mathbf{\bar{x}})^T\\
         \mathbf{\bar{x}} &= \frac{1}{N}\sum_{i=0}^{N-1} \mathbf{x}_i
\end{align}$$
     
 and $\mathbf{w}_{1}\in \mathbb{R}^{D}$ is the projection vector we are looking for, $\mathbf{x}\in \mathbb{R}^{D}$ is one data sample, and $\mathbf{\bar{x}} = \tfrac{1}{N}\sum_{i=0}^{N-1}\mathbf{x}_i$ is the mean of the data.
 
 The solution to this problem consists in finding the eigenvector of data covariance matrix $\mathbf{C}$ with the largest eigenvalue. To project to $d\geq1$ dimensions, one take the $d(\leq D)$ eigenvectors with largest eigenvalues and aggregates them into a matrix $\mathbf{W} = [\mathbf{w}_{1}, \mathbf{w}_{2}, ..., \mathbf{w}_{d} ]$. Hence, $\mathbf{W}$ is a matrix of $d$ eigenvectors each being $D$-dimensional. 
 
Once $\mathbf{W}$ has been found, we can project our original data $\mathbf{X}\in \mathbb{R}^{N\times D}$ to $\mathbf{Y}\in \mathbb{R}^{N\times d}$, using the centered data $\tilde{\mathbf{X}}\in R^{N\times D}$,
$$\begin{align}
        \mathbf{Y} &= \mathbf{\tilde{X}}\mathbf{W} \\
        \tilde{\mathbf{x}}_i &= \mathbf{x}_i-\mathbf{\bar{x}} ~~~~ \text{for } 0 \leq i \leq N-1
\end{align}$$
 
Finally, to understand how much of the variance is explained by our $d$ eigenvectors, we compute the percentage of the variance explained as 
$$\begin{align}
        \mathbf{exvar} = \frac{\sum_{i=0}^{d-1}\lambda_i}{\sum_{i=0}^{D-1}\lambda_i}
\end{align}$$
where $\lambda_i$ is the ith largest eigenvalue. For different applications, one would like to choose $d$ such that the explained variance is greater than a threshold.

You are asked to code the ```PCA``` that implements the above procedure.

In [ ]:
def PCA(X, d):
    '''
    Input:
        X: NxD matrix representing our data
        d: Number of principal components to be used to reduce dimensionality
        
    Output:
        mean_data: 1xD representing the mean of the input data
        W: Dxd matrix representing the principal components
        eg: d values representing the variance corresponding to the principal components, ie. the eigenvalues
        Y: Nxd data projected in the principal components' direction
        exvar: explained variance by the principal components
    '''
    ### WRITE YOUR CODE BELOW ###
    # Compute the mean of data
    mean = ...
    # Center the data with the mean
    X_tilde = ...
    # Create the covariance matrix
    C = ...
    # Compute the eigenvectors and eigenvalues. Hint: look into np.linalg.eigh()
    eigvals, eigvecs = ...
    # Choose the top d eigenvalues and corresponding eigenvectors. 
    # Hint: sort the eigenvalues (with corresponding eigenvectors) in decreasing order first.
    eigvals = ...
    eigvecs = ...

    W = ...
    eg = ...

    # project the data using W
    Y = ...
    
    # Compute the explained variance
    exvar = ...

    return mean, W, eg, Y, exvar

Let's call the above function and visualize the projected data

In [ ]:
d = 2
mean, W, eg, Y, exvar = PCA(data, d)
print(f'The total variance explained by the first {d} principal components is {exvar:.3f} %')

assert np.isclose(exvar, 97.77, atol=0.01), 'The explained variance is not correct.'

In [ ]:
plt.figure()
for ind,name in enumerate(cls_names):
    filtered_class = labels==ind
    plt.scatter(Y[filtered_class,0], Y[filtered_class,1], c=colors[ind,None], label=name)
plt.xlabel(f'feature_0')
plt.ylabel(f'feature_1')
plt.legend()

For PCA and generally speaking:

**Q.** What happens when $d=D$?

**A.** There is no loss of information, but the data is projected to uncorrelated axes. (Imagine if the data was distributed in an ellipsoid, then after PCA with $d=D$, the ellipsoid would be axis-aligned so that its covariance matrix is diagonal.)

**Q.** What happens when $D\gg N$?

**A.** Most of the eigenvalues will be zero as it means that most of the data reside in a subspace of dimension at maximum $N-1$. Additionally, since the computation complexity is ~$O(D^3)$, the current implementation will be slow.    
For more explanations see Bishop on pages 569-570.

# 3 EigenFaces
Now, we will use PCA on images of faces. The goal is to represent faces in the dataset as a linear combination of so-called *eigenfaces*, i.e., eigenvectors of this dataset of faces that can be visualize as images.

**Q.** Why can the eigenvectors be represented as images of faces?

**A.** The eigenvectors are of dimension $D$, like the data. So they can be reshaped and shown as images as well.

In [ ]:
faces = fetch_olivetti_faces().data
print(f'Dimensions of the Face dataset: N={faces.shape[0]}, D={faces.shape[1]}')

Run PCA on this dataset, and try different values of $d$ to see the impact on the explained variance.

In [ ]:
d = 30
mean, W, eg, Y, exvar = PCA(faces, d)
print(f'The total variance explained by the first {d} principal components is {exvar:.3f} %')

## 3.1 Visualize
Let us see what the mean face and the principal components look like.

In [ ]:
plt.figure()
plt.imshow(mean.reshape(64,64), cmap='gray')
plt.title('Mean Face');

In [ ]:
# Visualize the first 10 principal components
plt.figure(figsize=(8,18))
for i in range(10):
    plt.subplot(5,2,i+1)
    plt.imshow(W.reshape(64,64,-1)[...,i], cmap='gray')
    plt.title(f'Principal Component:{i}')

Observe what these components account for by adding/substracting them to the mean.

In [ ]:
# Choose the index of the component you want to visualize, from 0 to d-1
component_index = 0

# We add the component times the weight to the mean.
component_weights = [-5, -2, 0, 2, 5]

fig, axs = plt.subplots(1, 5, figsize=(20, 4))
fig.suptitle(f'Principal Component {component_index}')
for i, ax in enumerate(axs):
    component_weight = component_weights[i]
    # We build the image as the mean moved in the direction of a principal component
    img = mean + W.copy()[:, component_index] * component_weight
    ax.imshow(img.reshape(64, 64), cmap='gray')
    ax.set_title(f'weight {component_weight}')
fig.show();

**Q.** Can you identify what component accounts for what?

**A.** The first seem to be a mix of illumination and age, the second illumination from left/right, the third changes the face width, and others, such as the 12th(0-indexed), vary from a smiling to a neutral face. 

While it is interesting to see what PCA finds on the principal components, we should note that these are just our interpretation as PCA has no semantic information and simply tries to look for the largest variances in the data.

## 3.2 Reconstruction
We can now project one original data sample $\mathbf{x}_i \in \mathbb{R}^{D}$ to a lower-dimensional representation $\mathbf{y}_i  \in \mathbb{R}^{d} $ using $\mathbf{W} \in \mathbb{R}^{D\times d}$ and $\bar{\mathbf{x}} \in \mathbb{R}^{D}$ using the following operation: 
$$\mathbf{y}_i = \mathbf{W}^\top (\mathbf{x}_i - \bar{\mathbf{x}})$$

From this compressed representation $\mathbf{y}_i$, we can recover an approximation of the original data $\hat{\mathbf{x}}_i \in \mathbb{R}^{D}$ by using the opposite projection:
$$\hat{\mathbf{x}}_i = \bar{\mathbf{x}} + \mathbf{W}\mathbf{y}_i$$


Depending on how many dimension $d$ are kept, we will have some loss of information. Here we will see how changing $d$ affects the reconstruction $\hat{\mathbf{x}}_i$.

In [ ]:
# Try different values of d
d = 10
mean, W, eg, Y, exvar = PCA(faces, d)
print(f'The total variance explained by the first {d} principal components is {exvar:.3f} %')

In [ ]:
# Here we first select a random face from the dataset
sample_id = np.random.choice(faces.shape[0],1)[0]
sample_face = faces[sample_id,:]

### WRITE YOUR CODE HERE: project this face to its smaller dimension representation
proj_face = W.T @ (sample_face - mean)

### WRITE YOUR CODE HERE: undo the projection (by applying W.T), 
# to recover an approximation of the initial face, from proj_face
reconstructed_face = mean + W @ proj_face

In [ ]:
# We now visualize the original face, and the one reconstructed from
# the projection on the d first eigen vectors
plt.figure(figsize=(8,4))
plt.suptitle(f'Using d={d} dimensions')
ax = plt.subplot(1,2,1)
plt.imshow(sample_face.reshape(64,64),cmap='gray')
ax.set_title('Original Image')
ax = plt.subplot(1,2,2)
plt.imshow(reconstructed_face.reshape(64,64),cmap='gray')
ax.set_title('Reconstructed Image');

**Q.** The formulas above are for projecting and reconstructing a data point $\mathbf{x}_i$ using the principal component matrix $\mathbf{W} \in \mathbb{R}^{D\times d}$ and mean $\bar{\mathbf{x}} \in \mathbb{R}^{D}$. How can you rewrite these equations to project and reconstruct an entire dataset $\mathbf{X}\in\mathbb{R}^{N\times D}$ at once? *Hint*: thinking in term of matrix dimension can help you.

Assume we have $\bar{\mathbf{X}}\in \mathbb{R}^{1\times D}$, the mean of $\mathbf{X}$.

**A.** 
First, we project the data $\mathbf{X}$ to the lower dimensional space as
$$\mathbf{Y} = (\mathbf{X} - \bar{\mathbf{X}}) \mathbf{W}.$$

Then, we can reconstruct the data from it with
$$\hat{\mathbf{X}} = \bar{\mathbf{X}} + \mathbf{Y}\mathbf{W}^\top.$$

# 4 Fisher Linear Discriminat Analysis (LDA)
This supervised method is used to reduce dimensionality along with learning a projection, which keeps data points belonging to same class together. We will use `sklearn`'s implementation of the Fisher LDA to project MNIST data to smaller dimensions, and we will compare the results to PCA.

Note: we will work with a downsampled version of MNIST ($8\times8$ pixels) to speed things up.

In [ ]:
# Load MNIST data
mnist = datasets.load_digits()
data = mnist.data
labels = mnist.target
num_class = 10

We will project MNIST to a 2 and then a 3 dimensional space. This will allow us to visualize how the different digits are projected.

## 4.1 MNIST to 2D

In [ ]:
# Project with PCA
d = 2
mean, W, eg, Y, exvar = PCA(data, d)
print(f'Variance explained {exvar:.3f} % (PCA)')
proj_pca = Y

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Project with LDA
d = 2
lda = LDA(n_components=d)
# call the fit function
obj = lda.fit(data, labels)
# computed the variance explained using clf's parameter
exvar = lda.explained_variance_ratio_.sum() * 100
print(f'Variance explained {exvar:.3f} %')
proj = obj.transform(data)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(11, 5))
fig.suptitle('Projecting MNIST to 2D')
colors = colormaps["jet"](np.linspace(0, 1, num_class))
for i in range(num_class):
    inds = labels == i
    axs[0].scatter(proj_pca[inds,0], proj_pca[inds,1], color=colors[i])
    axs[1].scatter(proj[inds,0], proj[inds,1], color=colors[i])
axs[0].legend(np.arange(num_class))
axs[1].legend(np.arange(num_class))
axs[0].set_title('using PCA')
axs[1].set_title('using Fisher LDA')
fig.show();

**Q.** How are the classes after the projections? Which one would you prefer if you wanted to classify the results?

**A.** Because LDA uses the class labels to make the projection, it leads to a better class separation than PCA. So we would prefer to use LDA if we wanted to classify the data.

## 4.2 MNIST to 3D
Let's repeat that but in 3D.

In [ ]:
# Project with PCA
d = 3
mean, W, eg, Y, exvar = PCA(data, d)
print(f'Variance explained {exvar:.3f} % (PCA)')
proj_pca = Y

In [ ]:
# Project with LDA
d = 3
lda = LDA(n_components=d)
# call the fit function
obj = lda.fit(data, labels)
# computed the variance explained using clf's parameter
exvar = lda.explained_variance_ratio_.sum() * 100
print(f'Variance explained {exvar:.3f} %')
proj = obj.transform(data)

In [ ]:
fig = plt.figure(figsize=(11, 5))
fig.suptitle('Projecting MNIST to 3D')
axs = [plt.subplot(121, projection='3d'), plt.subplot(122, projection='3d')]
for i in range(num_class):
    inds = labels == i
    axs[0].scatter3D(proj_pca[inds,0], proj_pca[inds,1], proj_pca[inds,2], color=colors[i])
    axs[1].scatter3D(proj[inds,0], proj[inds,1], proj[inds,2], color=colors[i])
axs[0].legend(np.arange(num_class))
axs[1].legend(np.arange(num_class))
axs[0].set_title('using PCA')
axs[1].set_title('using Fisher LDA')
fig.show();

# 5 Written questions

**Q.1** (MCQ) Which of the following statements about PCA is/are true?
1. PCA is sensitive to initial values. A good practice is to run PCA several times with different random initializations.
2. We should normalize the data before running PCA on it.
3. PCA can be used to visualize high-dimensional data.
4. PCA can be used as a linear classifier.

**A.1** Correct answers are 2 and 3:
1. *False*. There is no random initialization for PCA. Running it multiple time will give the same results.
2. *True*. PCA looks for high variance in the data. If a feature has a very large magnitude compared to the others, then PCA will simply take it as the largest variance, which might overlook any relations between the features. Normalization before PCA is therefore good practice.
3. *True*. We projected MNIST to 2 and 3 dimension, which can then be simply plotted.
4. *False*. PCA does not act as a classifier and cannot make predictions. It can however be used to pre-process the data before another method for classification is applied.

**Q.2** Given the 2D data in the figure below, what are (approximatively) the first and second eigenvectors?

<img src="img/pca_q2.png" width="400">

**A.2** The eigenvectors are approximatively
$$\begin{align}
\mathbf{w}_1 &= \left(\begin{matrix} 0.90 \\ 0.43 \end{matrix}\right),\\
\mathbf{w}_2 &= \left(\begin{matrix} -0.43 \\ 0.90 \end{matrix}\right).
\end{align}$$

What's important here is to understand that $\mathbf{w}_1$ should align with the largest variance in the data, and $\mathbf{w}_2$ is orthogonal to it and both should be of norm $1$. Additionally, the sign is not important for the eigenvectors, for example $-\mathbf{w}_1$ could have also been the first eigenvector.

**Q.3** (True/False) If a binary classification problem is not linearly separable, we can apply PCA to project the data in a space where the problem will become linearly separable.

**A.3** *False*. PCA is a *linear* dimensionalty reduction technique that can at most rotate the data (and project onto a lower dimensional space). Such transformations cannot make a non-linearly separable problem into one. Note that LDA is also a linear technique, and thus could not make the problem linearly separable, even when using the class labels!

If we want to make it linearly separable, we should for example look into feature expansion (e.g., polynomial).

**Q.4** We have a dataset $\mathbf{X} \in \mathbb{R}^{N\times D}$ of $N$ samples of $D$ dimensions that we want to project to $d$ dimensions with PCA. What are the shapes of the data mean $\bar{\mathbf{X}}$, the covariance matrix $\mathbf{C}$, and the projection matrix $\mathbf{W}$?

**A.4**
* $\bar{\mathbf{X}}$ is of dimension $D$ (or $1\times D$ if we think in term of numpy array).
* $\mathbf{C}$ is of dimension $D\times D$.
* $\mathbf{W}$ is of dimension $D\times d$.